In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
from sklearn.datasets import fetch_lfw_pairs
lfw_pairs_train = fetch_lfw_pairs(subset='train', color=True, resize=1.)

lfw_pairs_train.pairs.shape

(2200, 2, 125, 94, 3)

In [ ]:
lfw_pairs_test = fetch_lfw_pairs(subset='test', color=True, resize=1.)

lfw_pairs_test.pairs.shape

(1000, 2, 125, 94, 3)

In [ ]:
lfw_pairs_test.pairs = lfw_pairs_test.pairs.astype('float32')
lfw_pairs_train.pairs = lfw_pairs_train.pairs.astype('float32')
lfw_pairs_test.pairs /= 255
lfw_pairs_train.pairs /= 255

In [ ]:
input_shape = 125, 94, 3

In [ ]:
epochs = 10

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):

    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def create_base_network(input_shape):

    input = Input(shape=input_shape)
    x = Conv2D(32, (5, 5), padding='same', activation='relu')(input)
    x = MaxPool2D((2, 2), (2, 2))(x)
    x = Conv2D(64, (5, 5), padding='same', activation='relu')(x)
    x = MaxPool2D((2, 2), (2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

def compute_accuracy(y_true, y_pred):

    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

def accuracy(y_true, y_pred):

    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([lfw_pairs_train.pairs[:, 0], lfw_pairs_train.pairs[:, 1]], lfw_pairs_train.target,
          batch_size=128,
          epochs=epochs,
          validation_data=([lfw_pairs_test.pairs[:, 0], lfw_pairs_test.pairs[:, 1]], lfw_pairs_test.target))

y_pred = model.predict([lfw_pairs_train.pairs[:, 0], lfw_pairs_train.pairs[:, 1]])
tr_acc = compute_accuracy(lfw_pairs_train.target, y_pred)
y_pred = model.predict([lfw_pairs_test.pairs[:, 0], lfw_pairs_test.pairs[:, 1]])
te_acc = compute_accuracy(lfw_pairs_test.target, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Train on 2200 samples, validate on 1000 samples
Epoch 1/10
2200/2200 [==============================] - 47s 21ms/step - loss: 423.1533 - accuracy: 0.5288 - val_loss: 0.3834 - val_accuracy: 0.4883
Epoch 2/10
2200/2200 [==============================] - 44s 20ms/step - loss: 0.2694 - accuracy: 0.4951 - val_loss: 0.3814 - val_accuracy: 0.4883
Epoch 3/10
2200/2200 [==============================] - 48s 22ms/step - loss: 0.2670 - accuracy: 0.5297 - val_loss: 0.4271 - val_accuracy: 0.4883
Epoch 4/10
2200/2200 [==============================] - 48s 22ms/step - loss: 0.2782 - accuracy: 0.5049 - val_loss: 0.4065 - val_accuracy: 0.4883
Epoch 5/10
2200/2200 [==============================] - 47s 21ms/step - loss: 0.3138 - accuracy: 0.5017 - val_loss: 0.3442 - val_accuracy: 0.5002
Epoch 6/10
2200/2200 [==============================] - 43s 20ms/step - loss: 0.2912 - accuracy: 0.5019 - val_loss: 0.3653 - val_accuracy: 0.4883
Epoch 7/10
2200/2200 [==============================] - 43s 20ms/step - lo